# MOFs LM Probing for hasMetals

In [ ]:
from baseline import PromptSet, logger as baseline_logger

from tqdm.auto import tqdm
from nltk.corpus import stopwords

import pandas as pd

from transformers import AutoTokenizer, AutoModelForMaskedLM, pipeline, logging

logging.set_verbosity_error()  # avoid irritating transformers warnings

## Read in the Triples

In [ ]:
file = "../data/mof_metals.csv"

In [ ]:
df = pd.read_csv(file)
df.head()

In [ ]:
df.shape

## Probing BERT

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-large-cased")
model = AutoModelForMaskedLM.from_pretrained("bert-large-cased")

In [ ]:
mask_token = tokenizer.mask_token
mask_token

## Creat the Prompt Pipeline

In [ ]:
pipe = pipeline(
    task="fill-mask",
    model=model,
    tokenizer=tokenizer,
    top_k=100,
)

## Create Prompts

In [ ]:
def create_metal_prompt1(sub, rel, mask):
    if rel == 'hasMetals':
        return f"{sub} contains {mask} which is an metal."
def create_metal_prompt2(sub, rel, mask):
    if rel == 'hasMetals':
        return f"{sub} contains an metal which is {mask}."
def create_metal_prompt3(sub, rel, mask):
    if rel == 'hasMetals':
        return f"{sub} is a metal organic framework. {sub} contains {mask} which is an metal."
def create_metal_prompt4(sub, rel, mask):
    if rel == 'hasMetals':
        return f"As a metal organic framework, {sub} contains an metal which is {mask}."
def create_metal_prompt5(sub, rel, mask):
    if rel == 'hasMetals':
        return f"{sub} is a MOF. {sub} has SBU metal {mask}."
def create_metal_prompt6(sub, rel, mask):
    if rel == 'hasMetals':
        return f"{sub} is MOF. {sub} contains {mask}." 
def create_metal_prompt7(sub, rel, mask):
    if rel == 'hasMetals':
        return f"{sub} contains {mask}."
def create_metal_prompt8(sub, rel, mask):
    if rel == 'hasMetals':
        return f"{sub} is a metal organic framework. {sub} contains {mask}."
def create_metal_prompt9(sub, rel, mask):
    if rel == 'hasMetals':
        return f"The SBU of {sub} is {mask}."
def create_metal_prompt10(sub, rel, mask):
    if rel == 'hasMetals':
        return f"{sub} is a metal organic framework structure composed of metal cluster {mask}."
def create_metal_prompt11(sub, rel, mask):
    if rel == 'hasMetals':
        return f"{sub} is a metal organic framework. {sub} has SBU metal {mask}."
def create_metal_prompt12(sub, rel, mask):
    if rel == 'hasMetals':
        return f"{mask} was used as metal center in the synthesized {sub} material."

In [ ]:
processors = [create_metal_prompt1, create_metal_prompt2,
              create_metal_prompt3, create_metal_prompt4,
              create_metal_prompt5, create_metal_prompt6,
              create_metal_prompt7, create_metal_prompt8,
              create_metal_prompt9, create_metal_prompt10,
              create_metal_prompt11, create_metal_prompt12
             ]

In [ ]:
def prompt_probe(model, tokenizer, prompt_processor, df_sub, mask_token, \
                 subjectCol, relationCol, objectCol, top_k=100):

    pipe = pipeline(
        task="fill-mask",
        model=model,
        tokenizer=tokenizer,
        top_k=top_k,
    )

    prompts = PromptSet([prompt_processor(row.SubjectEntity, row.Relation, mask_token) for _, row in df_sub.iterrows()])

    outputs = []
    for out in tqdm(pipe(prompts, batch_size=8), total=len(prompts)):
        outputs.append(out)

    results = []
    for idx, row in df_sub.iterrows():
        for output in outputs[idx]:
            record = {}
            record['SubjectEntity'] = row[subjectCol]
            record['Relation'] = row[relationCol]
            record['ObjectEntity'] = row[objectCol]
            record['prompt']= prompts[idx]
            record['predictedScore'] = output['score']
            record['predictedToken'] = output['token_str']
            results.append(record)

    return pd.DataFrame(results)

In [ ]:
# Compute a column indicating whether the predictedToken in the ObjectEntity list
def hasPredicted(row):
    token = row['predictedToken']
    objectEntities = row['ObjectEntity']
    #print(objectEntities)
    if pd.notna(objectEntities):
        objectList = objectEntities.split(",")
        if token in objectList:
            return 1
        else:
            return 0
    else:
        return 0

In [ ]:
# return the top-k rows based on predictedScore
def top(df, col, n=10):
    return df.sort_values(by=col, ascending=False)[:n]

In [ ]:
# check whether the predicted is 1 for each group
def predictedOne(df, col):
    total = df[col].sum()
    if total > 0:
        return 1
    else:
        return 0

In [ ]:
# Assume the predicted_df has a binary column 'predicted' indicating whether ObjectEntity has been predicted
# 1 for yes, 0 for no
def evaluate_acc_n(predicted_df, relation, prompt_template, at_k=5):
    topk_df = predicted_df.groupby('SubjectEntity', as_index=False).apply(top, col='predictedScore', n=at_k)
    topk_agg_df = topk_df.groupby('SubjectEntity', as_index=False).apply(predictedOne, col='predicted')
    topk_agg_df.columns = ['SubjectEntity', 'predicted']
    result = {}
    result['Relation'] = relation
    result['prompt_template'] = prompt_template
    result['acc'] = topk_agg_df.predicted.mean()
    result['at_k'] = at_k

    return result

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
%%time
acc_list = []
atks = [1, 5, 10, 20, 50, 100]
for processor in processors:
    result_df = prompt_probe(model, tokenizer, processor, df, mask_token, \
                     'SubjectEntity', 'Relation', 'ObjectEntityName', top_k=100)
    result_df['predicted'] = result_df.apply(hasPredicted, axis=1)

    prompt_template = processor('{SubjectEntity}', 'hasMetals', '[MASK]')
    for at_k in atks:
        acc_df = evaluate_acc_n(result_df, 'hasMetals', prompt_template, at_k=at_k)
        acc_list.append(acc_df)

In [ ]:
result_df.head()

In [ ]:
result_df.shape

In [ ]:
result_df.to_csv("../data/predicted_hasMetals_BERT.csv", index=None)

In [ ]:
hasMetals_df = pd.DataFrame(acc_list)

In [ ]:
hasMetals_df.sample(10)

In [ ]:
hasMetals_df.to_csv("../data/predicted_hasMetals_BERT_eval.csv", index=None)

In [ ]:
hasMetals_df[hasMetals_df['at_k'] == 1].sort_values('acc', ascending=False)

In [ ]:
hasMetals_df.sort_values('acc', ascending=False)

## Probing MatBERT

In [ ]:
# MatBERT
tokenizer = AutoTokenizer.from_pretrained("../model/model/matbert-base-cased")
model = AutoModelForMaskedLM.from_pretrained("../model/model/matbert-base-cased")

In [ ]:
mask_token = tokenizer.mask_token
mask_token

## Creat the Prompt Pipeline

In [ ]:
pipe = pipeline(
    task="fill-mask",
    model=model,
    tokenizer=tokenizer,
    top_k=100,
)

## Run

In [ ]:
%%time
acc_list = []
atks = [1, 5, 10, 20, 50, 100]
for processor in processors:
    result_df = prompt_probe(model, tokenizer, processor, df, mask_token, \
                     'SubjectEntity', 'Relation', 'ObjectEntityName', top_k=100)
    result_df['predicted'] = result_df.apply(hasPredicted, axis=1)

    prompt_template = processor('{SubjectEntity}', 'hasMetals', '[MASK]')
    for at_k in atks:
        acc_df = evaluate_acc_n(result_df, 'hasMetals', prompt_template, at_k=at_k)
        acc_list.append(acc_df)

In [ ]:
hasMetals_df = pd.DataFrame(acc_list)

In [ ]:
hasMetals_df.sample(10)

In [ ]:
result_df.to_csv("../data/predicted_hasMetals_MatBERT.csv", index=None)

In [ ]:
hasMetals_df.to_csv("../data/predicted_hasMetals_MatBERT_eval.csv", index=None)

In [ ]:
hasMetals_df[hasMetals_df['at_k'] == 1].sort_values('acc', ascending=False)

In [ ]:
hasMetals_df.sort_values('acc', ascending=False)

## Probing MatSciBERT

In [ ]:
# MatSciBERT
tokenizer = AutoTokenizer.from_pretrained('m3rg-iitd/matscibert')
model = AutoModelForMaskedLM.from_pretrained('m3rg-iitd/matscibert')

In [ ]:
mask_token = tokenizer.mask_token
mask_token

## Creat the Prompt Pipeline

In [ ]:
pipe = pipeline(
    task="fill-mask",
    model=model,
    tokenizer=tokenizer,
    top_k=100,
)

## Run

In [ ]:
%%time
acc_list = []
atks = [1, 5, 10, 20, 50, 100]
for processor in processors:
    result_df = prompt_probe(model, tokenizer, processor, df, mask_token, \
                     'SubjectEntity', 'Relation', 'ObjectEntityName', top_k=100)
    result_df['predicted'] = result_df.apply(hasPredicted, axis=1)

    prompt_template = processor('{SubjectEntity}', 'hasMetals', '[MASK]')
    for at_k in atks:
        acc_df = evaluate_acc_n(result_df, 'hasMetals', prompt_template, at_k=at_k)
        acc_list.append(acc_df)

In [ ]:
hasMetals_df = pd.DataFrame(acc_list)

In [ ]:
hasMetals_df.sample(10)

In [ ]:
result_df.to_csv("../data/predicted_hasMetals_MatSciBERT.csv", index=None)

In [ ]:
hasMetals_df.to_csv("../data/predicted_hasMetals_MatSciBERT_eval.csv", index=None)

In [ ]:
hasMetals_df[hasMetals_df['at_k'] == 1].sort_values('acc', ascending=False)

In [ ]:
hasMetals_df.sort_values('acc', ascending=False)

## Probing SciBERT

In [ ]:
# SciBERT
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_cased')
model = AutoModelForMaskedLM.from_pretrained('allenai/scibert_scivocab_cased')

In [ ]:
mask_token = tokenizer.mask_token
mask_token

## Creat the Prompt Pipeline

In [ ]:
pipe = pipeline(
    task="fill-mask",
    model=model,
    tokenizer=tokenizer,
    top_k=100,
)

## Run

In [ ]:
%%time
acc_list = []
atks = [1, 5, 10, 20, 50, 100]
for processor in processors:
    result_df = prompt_probe(model, tokenizer, processor, df, mask_token, \
                     'SubjectEntity', 'Relation', 'ObjectEntityName', top_k=100)
    result_df['predicted'] = result_df.apply(hasPredicted, axis=1)

    prompt_template = processor('{SubjectEntity}', 'hasMetals', '[MASK]')
    for at_k in atks:
        acc_df = evaluate_acc_n(result_df, 'hasMetals', prompt_template, at_k=at_k)
        acc_list.append(acc_df)

In [ ]:
hasMetals_df = pd.DataFrame(acc_list)

In [ ]:
hasMetals_df.sample(10)

In [ ]:
result_df.to_csv("../data/predicted_hasMetals_SciBERT.csv", index=None)

In [ ]:
hasMetals_df.to_csv("../data/predicted_hasMetals_SciBERT_eval.csv", index=None)

In [ ]:
hasMetals_df[hasMetals_df['at_k'] == 1].sort_values('acc', ascending=False)

In [ ]:
hasMetals_df.sort_values('acc', ascending=False)

## Probing RoBERTa

In [ ]:
# RoBERTa
tokenizer = AutoTokenizer.from_pretrained("roberta-large")
model = AutoModelForMaskedLM.from_pretrained("roberta-large")

In [ ]:
mask_token = tokenizer.mask_token
mask_token

## Creat the Prompt Pipeline

In [ ]:
pipe = pipeline(
    task="fill-mask",
    model=model,
    tokenizer=tokenizer,
    top_k=100,
)

## Run

In [ ]:
%%time
acc_list = []
atks = [1, 5, 10, 20, 50, 100]
for processor in processors:
    result_df = prompt_probe(model, tokenizer, processor, df, mask_token, \
                     'SubjectEntity', 'Relation', 'ObjectEntityName', top_k=100)
    result_df['predicted'] = result_df.apply(hasPredicted, axis=1)

    prompt_template = processor('{SubjectEntity}', 'hasMetals', '[MASK]')
    for at_k in atks:
        acc_df = evaluate_acc_n(result_df, 'hasMetals', prompt_template, at_k=at_k)
        acc_list.append(acc_df)

In [ ]:
hasMetals_df = pd.DataFrame(acc_list)

In [ ]:
hasMetals_df.sample(10)

In [ ]:
result_df.to_csv("../data/predicted_hasMetals_RoBERTa.csv", index=None)

In [ ]:
hasMetals_df.to_csv("../data/predicted_hasMetals_RoBERTa_eval.csv", index=None)

In [ ]:
hasMetals_df[hasMetals_df['at_k'] == 1].sort_values('acc', ascending=False)

In [ ]:
hasMetals_df.sort_values('acc', ascending=False)